# 1. IMPORT & SETUP

In [2]:
# agent & model
from agno.agent import Agent
from agno.models.anthropic import Claude
import asyncio

# tools
from agno.tools import tool
from agno.tools.github import GithubTools

# mcp
from agno.tools.mcp import MCPTools
from agno.tools.mcp import MultiMCPTools

# memory management in sqlite
from agno.db.sqlite import SqliteDb

# env to load secrets
import os
import dotenv
dotenv.load_dotenv()

True

# 2. BASIC AGENT

In [3]:
# setting up the agent with configs
basic_agent = Agent(
    model = Claude(id="claude-sonnet-4-5"), ## model
    instructions="You are a helpful assistant", ## system prompt
    markdown = True, ## respond in markdown
    stream = True, ## allow model to stream responses as it goes
)

In [4]:
# call the agent
basic_agent.print_response(
    input(), 
    stream=True
)

Output()

# 3. AGENT WITH TOOL CALL

## 3.1. Hard-coded tool

In [5]:
from agno.tools import tool

let's create a tool to return power minus one of two integers:

power_one(x, y)

result = x*y - 1

In [16]:
# @tool
def power_one(x: int, y: int) -> int:
    """Returns power minus one of two integers."""
    return x**y - 1

# let's test it

power_one(3, 4) # should return 3*4 - 1 = 81 - 1 = 80

80

Now, in order to turn it into a tool for agent, a "@tool" must be added to the top

In [17]:
@tool
def power_one(x: int, y: int) -> int:
    """Returns x to the power of y minus one."""
    return x**y - 1

Now we can add this tool to our agent

In [18]:
tool_agent = Agent(
    model = Claude(id="claude-sonnet-4-5"), ## model
    instructions="You are a helpful assistant", ## system prompt
    markdown = True, ## respond in markdown
    stream = True, ## allow model to stream responses as it goes
    tools = [power_one] ## add the tool here
)

In [19]:
# let's call the agent

tool_agent.print_response(
    input(),
    stream=True
)

Output()

## 3.2. Agno available ToolKit

First, we need to import the tools package. You can find all of them here: https://docs.agno.com/concepts/tools/toolkits/toolkits

Let's use the Github tool from Agno. You'd need to go to Github -> Settings -> Developer Settings -> Personal Access Tokens -> Generate New Token.


Generate a GitHub Personal Access Token:
1. Go to Github → Settings → Developer Settings → Personal Access Tokens → Generate New Token
2. Copy the token to your .env file as `GITHUB_ACCESS_TOKEN="your_token_here"`

Run this:
```Python
pip install -U PyGithub
```

In [20]:
import dotenv
dotenv.load_dotenv()

True

In [22]:
# import Github tool

from agno.tools.github import GithubTools

In [25]:
# just add tool to our agent, it will handle auth with env automatically

github_agent = Agent(
    model=Claude(id="claude-sonnet-4-5"), ## model
    instructions="You are a helpful assistant who can fetch information about a user's Github.", ## system prompt
    markdown=True, ## respond in markdown
    stream=True, ## allow model to stream responses as it goes
    tools=[GithubTools()] ## add the tool here
)

In [26]:
prompt = "What are my personal repositories on Github? Can you tell from those what I do?"

github_agent.print_response(
    prompt,
    stream=True
)

Output()

## 3.3. MCP Servers

Let's test with 2 types of MCP installations:
- Locally installed MCP (in this case Freepik and DataForSEO)
- Hosted MCP server (int his case Brave Search)

In [27]:
from agno.tools.mcp import MultiMCPTools ## for MCP handling
import asyncio ## for async calls to agent

In [28]:
async def mcp_agent(message: str) -> None:
    """Run the Agent with given message"""

    env = {
        **os.environ,
        'FREEPIK_API_KEY': os.getenv('FREEPIK_API_KEY'),
        'DATAFORSEO_USERNAME': os.getenv('DATAFORSEO_USERNAME'),
        'DATAFORSEO_PASSWORD': os.getenv('DATAFORSEO_PASSWORD'),
        'BRAVE_API_KEY': os.getenv('BRAVE_API_KEY'),
    } ## Configure API keys

    ## CONNECT MCP
    mcp_tools = MultiMCPTools(
        commands=[
            "node D:/MCP/mcp-servers/freepik-mcp/build/index.js",
            "node D:/MCP/mcp-server-typescript/build/main/main/index.js",
            "npx -y @brave/brave-search-mcp-server"
        ],
        env=env
    )

    await mcp_tools.connect()

    try:
        agent = Agent(
            tools=[mcp_tools],
            markdown=True,
        )

        await agent.aprint_response(message, stream=True)
    
    finally:
        await mcp_tools.close()

Let's give it a try with different prompts to test the MCP servers

In [29]:
prompt_freepik = "Find me 3 free images of cute real ducks."
prompt_dataforseo = "Give me a short on page of this website: 'https://seongon.com'"
prompt_brave = "what is the current scores between Real Madrid and Barcelona all time?"

In [ ]:
## call the agent
## ! IMPORTANT: CAN ONLY RUN CODE IN A PYTHON FILE, NOT A NOTEBOOK FILE

if __name__ == "__main__":
    asyncio.run(
        mcp_agent(
            "What will be the next Portugal match?"
        )
    )